In [1]:
import pandas as pd
import time
import re
import tweepy
import requests
from bs4 import BeautifulSoup
from types import MappingProxyType
import datetime
import math
import random


# 2022-23のプレミアリーグクラブ別トップスコアラー（英語 ⇒ 選手詳細URL）
EPL_TOP_TEAM_SCORER_DICT: dict = MappingProxyType({
        'Erling Haaland':'https://fbref.com/en/players/1f44ac21/goallogs/dom_lg/Erling-Haaland-Goal-Log',
})

In [2]:
def scrapeFbrefHomePage():
    df = pd.DataFrame()

    # 1秒スリープ
    time.sleep(1)

    # スクレイピング対象のURLを生成する
    url = 'https://fbref.com/en/comps/9/Premier-League-Stats'
    df = pd.read_html(url)[0]
    
    # マルチインデックスを解消する
    df = df.T.reset_index(level=0, drop=True).T
    html = requests.get(url)
    html.encoding = "EUC-JP"
    
    # HTMLを解析する
    soup = BeautifulSoup(html.text, "lxml")
    
    # リーグ順位欄と最多得点者を表示する
    df = df.drop([13,14,15,16,18,19], axis=1)
    
    # Top Team Scorer
    top_team_scorer_list = []
    for num in range(len(df[17])):
        top_team_scorer_list.append(df[17][num])
    
    
    return df, top_team_scorer_list

In [6]:
fbrefClubResult = scrapeFbrefHomePage()
df = fbrefClubResult[0]

In [4]:
def scrapeFbrefPlayerPage():
    df_player = pd.DataFrame()

    # 1秒スリープ
    time.sleep(1)

    # スクレイピング対象のURLを生成する
    url = 'https://fbref.com/en/players/1f44ac21/goallogs/dom_lg/Erling-Haaland-Goal-Log'
    df_player = pd.read_html(url)[0]
    
    # マルチインデックスを解消する
    df_player = df_player.T.reset_index(level=0, drop=True).T
    html = requests.get(url)
    html.encoding = "EUC-JP"
    
    # HTMLを解析する
    soup = BeautifulSoup(html.text, "lxml")
    
    # クラブ名を補完していく
    score_club_list = []
    for num in range(21): # 選手横の数値
        score_club_list.append(df_player[6][num])
    print(score_club_list)
        
    return score_club_list;

In [5]:
fbrefPlayerResult = scrapeFbrefPlayerPage()

['Everton', 'Leeds United', 'Leeds United', 'Fulham', 'Brighton', 'Brighton', 'Southampton', 'Manchester Utd', 'Manchester Utd', 'Manchester Utd', 'Wolves', 'Aston Villa', "Nott'ham Forest", "Nott'ham Forest", "Nott'ham Forest", 'Crystal Palace', 'Crystal Palace', 'Crystal Palace', 'Newcastle Utd', 'West Ham', 'West Ham']


In [10]:
df

# 色を変える関数を作成
## fbrefPlayerResultの要素数に応じてreturnするCSSを変える
## 列でクラブ名の文字列に一致するところだけ色を変えるように指定する

,0,1,2,3,4,5,6,7,8,9,10,11,12,17
0,1,Arsenal,17,14,2,1,40,14,26,44,2.59,32.3,15.0,"Martin Ødegaard, Martinelli - 7"
1,2,Manchester City,17,12,3,2,45,16,29,39,2.29,35.7,12.3,Erling Haaland - 21
2,3,Newcastle Utd,18,9,8,1,32,11,21,35,1.94,29.3,16.3,Miguel Almirón - 9
3,4,Manchester Utd,17,11,2,4,27,20,7,35,2.06,25.4,19.2,Marcus Rashford - 7
4,5,Tottenham,18,10,3,5,37,25,12,33,1.83,27.2,20.4,Harry Kane - 15
5,6,Liverpool,17,8,4,5,34,22,12,28,1.65,31.1,23.7,"Mohamed Salah, Roberto Firmino - 7"
6,7,Fulham,18,8,4,6,30,27,3,28,1.56,25.4,30.2,Aleksandar Mitrović - 11
7,8,Brighton,17,8,3,6,32,25,7,27,1.59,25.4,22.0,Leandro Trossard - 7
8,9,Brentford,18,6,8,4,30,28,2,26,1.44,25.6,24.6,Ivan Toney - 12
9,10,Chelsea,17,7,4,6,20,19,1,25,1.47,18.6,23.3,"Raheem Sterling, Kai Havertz - 4"


In [25]:
def color_background_lightgreen(fbrefPlayerResult):
    for num in range(len(fbrefPlayerResult)):
        print(fbrefPlayerResult[num])
        
        
    color = 'lightgreen' if fbrefPlayerResult[num] == 'Everton' else ''
    return 'background-color: %s' % color

    #表示
    df.style.applymap(color_background_lightgreen)
    
    print(df)

In [34]:
# マイナスの値を赤くする
def color_negative_red(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: red'` for negative
    strings, black otherwise.
    """
    color = 'red' if val == 'Arsenal' else 'black'
    return 'color: %s' % color

In [36]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,17
0,1,Arsenal,17,14,2,1,40,14,26,44,2.59,32.3,15.0,"Martin Ødegaard, Martinelli - 7"
1,2,Manchester City,17,12,3,2,45,16,29,39,2.29,35.7,12.3,Erling Haaland - 21
2,3,Newcastle Utd,18,9,8,1,32,11,21,35,1.94,29.3,16.3,Miguel Almirón - 9
3,4,Manchester Utd,17,11,2,4,27,20,7,35,2.06,25.4,19.2,Marcus Rashford - 7
4,5,Tottenham,18,10,3,5,37,25,12,33,1.83,27.2,20.4,Harry Kane - 15
5,6,Liverpool,17,8,4,5,34,22,12,28,1.65,31.1,23.7,"Mohamed Salah, Roberto Firmino - 7"
6,7,Fulham,18,8,4,6,30,27,3,28,1.56,25.4,30.2,Aleksandar Mitrović - 11
7,8,Brighton,17,8,3,6,32,25,7,27,1.59,25.4,22.0,Leandro Trossard - 7
8,9,Brentford,18,6,8,4,30,28,2,26,1.44,25.6,24.6,Ivan Toney - 12
9,10,Chelsea,17,7,4,6,20,19,1,25,1.47,18.6,23.3,"Raheem Sterling, Kai Havertz - 4"


In [ ]:
df.style.applymap(color_negative_red)